# Aula04.Ex02- Exemplo - Esfera - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [281]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [282]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Esfera", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação)
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação.

In [283]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [284]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [285]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [286]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [287]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [288]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [289]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [290]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando uma Esfera



In [291]:
import math


PI = 3.141592
r = 0.5 # raio
num_sectors = 30 # qtd de sectors (longitude)
num_stacks = 30 # qtd de stacks (latitude)

# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(PI)/num_stacks # variar de 0 até π

# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas na esfera
def F(u,v,r):
    x = r*math.sin(v)*math.cos(u)
    y = r*math.sin(v)*math.sin(u)
    z = r*math.cos(v)
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for i in range(0,num_sectors): # para cada sector (longitude)
    
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = PI
        else: vn = (j+1)*stack_step
        
        # verticies do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

In [292]:
for v in vertices:
    print(v)

([0. , 0. , 0.5],)
([0. , 0. , 0.5],)
([0.05226422, 0.        , 0.49726096],)
([0.05112212, 0.01086634, 0.49726096],)
([0.05226422, 0.        , 0.49726096],)
([0. , 0. , 0.5],)
([0.05226422, 0.        , 0.49726096],)
([0.05112212, 0.01086634, 0.49726096],)
([0.10395583, 0.        , 0.4890738 ],)
([0.10168414, 0.02161363, 0.4890738 ],)
([0.10395583, 0.        , 0.4890738 ],)
([0.05112212, 0.01086634, 0.49726096],)
([0.10395583, 0.        , 0.4890738 ],)
([0.10168414, 0.02161363, 0.4890738 ],)
([0.15450847, 0.        , 0.47552827],)
([0.15113209, 0.03212411, 0.47552827],)
([0.15450847, 0.        , 0.47552827],)
([0.10168414, 0.02161363, 0.4890738 ],)
([0.15450847, 0.        , 0.47552827],)
([0.15113209, 0.03212411, 0.47552827],)
([0.20336828, 0.        , 0.45677274],)
([0.1989242 , 0.04228263, 0.45677274],)
([0.20336828, 0.        , 0.45677274],)
([0.15113209, 0.03212411, 0.47552827],)
([0.20336828, 0.        , 0.45677274],)
([0.1989242 , 0.04228263, 0.45677274],)
([0.24999996, 0.       

([-0. ,  0. ,  0.5],)
([0.00546311, 0.05197791, 0.49726096],)
([-0.00546308,  0.05197791,  0.49726096],)
([0.00546311, 0.05197791, 0.49726096],)
([-0. ,  0. ,  0.5],)
([0.00546311, 0.05197791, 0.49726096],)
([-0.00546308,  0.05197791,  0.49726096],)
([0.01086637, 0.10338634, 0.4890738 ],)
([-0.01086631,  0.10338635,  0.4890738 ],)
([0.01086637, 0.10338634, 0.4890738 ],)
([-0.00546308,  0.05197791,  0.49726096],)
([0.01086637, 0.10338634, 0.4890738 ],)
([-0.01086631,  0.10338635,  0.4890738 ],)
([0.01615058, 0.15366204, 0.47552827],)
([-0.01615048,  0.15366206,  0.47552827],)
([0.01615058, 0.15366204, 0.47552827],)
([-0.01086631,  0.10338635,  0.4890738 ],)
([0.01615058, 0.15366204, 0.47552827],)
([-0.01615048,  0.15366206,  0.47552827],)
([0.02125784, 0.2022542 , 0.45677274],)
([-0.0212577 ,  0.20225422,  0.45677274],)
([0.02125784, 0.2022542 , 0.45677274],)
([-0.01615048,  0.15366206,  0.47552827],)
([0.02125784, 0.2022542 , 0.45677274],)
([-0.0212577 ,  0.20225422,  0.45677274],)
([0

([-0.27066907,  0.1966528 , -0.37157226],)
([-0.3056407 ,  0.13608022, -0.37157226],)
([-0.23776421,  0.172746  , -0.40450835],)
([-0.2684844 ,  0.11953714, -0.40450835],)
([-0.23776421,  0.172746  , -0.40450835],)
([-0.3056407 ,  0.13608022, -0.37157226],)
([-0.23776421,  0.172746  , -0.40450835],)
([-0.2684844 ,  0.11953714, -0.40450835],)
([-0.20225437,  0.14694656, -0.43301257],)
([-0.22838652,  0.10168438, -0.43301257],)
([-0.20225437,  0.14694656, -0.43301257],)
([-0.2684844 ,  0.11953714, -0.40450835],)
([-0.20225437,  0.14694656, -0.43301257],)
([-0.22838652,  0.10168438, -0.43301257],)
([-0.16452858,  0.11953714, -0.45677263],)
([-0.1857864 ,  0.08271756, -0.45677263],)
([-0.16452858,  0.11953714, -0.45677263],)
([-0.22838652,  0.10168438, -0.43301257],)
([-0.16452858,  0.11953714, -0.45677263],)
([-0.1857864 ,  0.08271756, -0.45677263],)
([-0.12500018,  0.09081805, -0.47552818],)
([-0.14115076,  0.06284446, -0.47552818],)
([-0.12500018,  0.09081805, -0.47552818],)
([-0.185786

([-0.24863084, -0.43064037,  0.05226438],)
([-0.15366252, -0.4729231 ,  0.05226438],)
([-0.24863084, -0.43064037,  0.05226438],)
([-0.15113254, -0.46513665,  0.10395598],)
([-0.24863084, -0.43064037,  0.05226438],)
([-0.15366252, -0.4729231 ,  0.05226438],)
([-2.5000039e-01, -4.3301249e-01,  1.6339744e-07],)
([-1.5450893e-01, -4.7552812e-01,  1.6339744e-07],)
([-2.5000039e-01, -4.3301249e-01,  1.6339744e-07],)
([-0.15366252, -0.4729231 ,  0.05226438],)
([-2.5000039e-01, -4.3301249e-01,  1.6339744e-07],)
([-1.5450893e-01, -4.7552812e-01,  1.6339744e-07],)
([-0.24863085, -0.4306404 , -0.05226406],)
([-0.15366252, -0.47292313, -0.05226406],)
([-0.24863085, -0.4306404 , -0.05226406],)
([-1.5450893e-01, -4.7552812e-01,  1.6339744e-07],)
([-0.24863085, -0.4306404 , -0.05226406],)
([-0.15366252, -0.47292313, -0.05226406],)
([-0.2445373 , -0.42355016, -0.10395566],)
([-0.15113255, -0.46513674, -0.10395566],)
([-0.2445373 , -0.42355016, -0.10395566],)
([-0.15366252, -0.47292313, -0.05226406],)


([ 0.39566866, -0.28747082,  0.10395598],)
([ 0.33273208, -0.36953726,  0.05226438],)
([ 0.4022922 , -0.29228312,  0.05226438],)
([ 0.33273208, -0.36953726,  0.05226438],)
([ 0.39566866, -0.28747082,  0.10395598],)
([ 0.33273208, -0.36953726,  0.05226438],)
([ 0.4022922 , -0.29228312,  0.05226438],)
([ 3.3456489e-01, -3.7157279e-01,  1.6339744e-07],)
([ 4.0450814e-01, -2.9389310e-01,  1.6339744e-07],)
([ 3.3456489e-01, -3.7157279e-01,  1.6339744e-07],)
([ 0.4022922 , -0.29228312,  0.05226438],)
([ 3.3456489e-01, -3.7157279e-01,  1.6339744e-07],)
([ 4.0450814e-01, -2.9389310e-01,  1.6339744e-07],)
([ 0.3327321 , -0.3695373 , -0.05226406],)
([ 0.40229222, -0.29228315, -0.05226406],)
([ 0.3327321 , -0.3695373 , -0.05226406],)
([ 4.0450814e-01, -2.9389310e-01,  1.6339744e-07],)
([ 0.3327321 , -0.3695373 , -0.05226406],)
([ 0.40229222, -0.29228315, -0.05226406],)
([ 0.32725385, -0.36345306, -0.10395566],)
([ 0.39566872, -0.28747085, -0.10395566],)
([ 0.32725385, -0.36345306, -0.10395566],)


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [293]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [294]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [295]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [296]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [297]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [298]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [299]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [300]:
import math
import random
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    ### apenas para visualizarmos a esfera rotacionando
    d -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)     


    
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.